In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from dhis2 import Api

from src.main.common import FileUtil

In [ ]:
credentials  = FileUtil.load_credentias()
username = credentials['username']
password = credentials['password']

url = 'https://dhis-ao.icap.columbia.edu'
api = Api(url, username, password)

In [ ]:
# Get the list of supported facilities
orgUnit = 'UnCWi0cmwXy'

In [ ]:
def add_patient_demographics(patient, patient_demographic):
    
    patient_demographic = patient_demographic.json()

    for patient_demographic in patient_demographic['attributes']:
        # get patient ID
        if patient_demographic['attribute'] == 'UcHFpmknDEX':
            patient['patientIdentifier'] = patient_demographic['value']

        # get patient NAME
        if patient_demographic['attribute'] == 'sJk1y9bHXxj':
            patient['patientName'] = patient_demographic['value']

        # get patient AGE
        if patient_demographic['attribute'] == 'WHMtrrYH1WA':
            patient['patientAge'] = patient_demographic['value']
        
        # get patient GENDER
        if patient_demographic['attribute'] == 'DIcP7BuFpar':
            patient['patientSex'] = patient_demographic['value']

def add_consultation(enrollment, consultation):
    consultation = consultation.json()['instances'][0]
    
    for data in consultation['dataValues']:
        # get first consultation date 
        if data['dataElement'] == 'UydTSzRCUZe':
            enrollment['firstConsultationDate'] = data['value']
        
        # ART start date
        if data['dataElement'] == 'QUgeSSXNzQB':
            enrollment['artStartDate'] = data['value']

def add_pharmacy(enrollment, pharmacy):
    if pharmacy.json()['instances']:
        pharmacy = pharmacy.json()['instances'][0]
        
        for data in pharmacy['dataValues']:
            # get last pickup date 
            if data['dataElement'] == 'f1E1lgVGby4':
                enrollment['lastPickupDate'] = data['value']
            
            # drugs pickup quantity
            if data['dataElement'] == 'PWY0qgJN37G':
                enrollment['pickupQuantity'] = data['value']
        
        #calculate next pickupDate 
        enrollment['nextPickupDate'] = pd.to_datetime(enrollment['lastPickupDate']) + pd.Timedelta(days=int(enrollment['pickupQuantity']))

def add_laboratory(enrollment, lab):
    if lab.json()['instances']:
        lab = lab.json()['instances'][0]
        
        for data in lab['dataValues']:
            # get last CD4 value 
            if data['dataElement'] == 'mHMBFxYe46z':
                enrollment['lastCD4'] = data['value']

def add_final_outcome(enrollment, final_outcome):
    if final_outcome.json()['instances']:

        final_outcome = final_outcome.json()['instances'][0]
        
        for data in final_outcome['dataValues']:
            # Transfered out
            if data['dataElement'] == 'JfsZJNCPYMy':
                enrollment['transferedOut'] = data['value']
            
            # date of transfer
            if data['dataElement'] == 'tlBekbbYASc':
                enrollment['dateOfTransfer'] = data['value']

            # dead
            if data['dataElement'] == 'ZmoHwddq6DD':
                enrollment['dead'] = data['value']

            # date of death
            if data['dataElement'] == 'KmRHXvsQy2W':
                enrollment['dateOfDeath'] = data['value']

In [5]:
# TX_CURR indicator
# Care & treatment Program
program = 'YKs1kXOLb0u'

#get all enrollments
patients = api.get('tracker/enrollments', params={'orgUnit':orgUnit, 'skipPaging':'true', 'program': program, 'fields':'{,enrollment, enrolledAt, trackedEntity, program, status,}'})
patients = patients.json()['instances']

for patient in patients:
    # get patient
    tracked_entity = patient['trackedEntity']
    patient_demographic = api.get(f'tracker/trackedEntities/{tracked_entity}', params={'orgUnit':orgUnit, 'trackedEntityType':'F3jh38FNCkP', 'skipPaging':'true','fields':'{,trackedEntity,trackedEntityType, attributes=[attribute, displayName,value]}'})
    add_patient_demographics(patient, patient_demographic)

    # get first consultation
    consultation_stage = 'SvhTuAlw3zl'
    first_consultation_element_id = 'UydTSzRCUZe'
    first_consultation = api.get('tracker/events', params={'orgUnit':'UnCWi0cmwXy', 'program':program, 'programStage':consultation_stage, 'trackedEntity':tracked_entity, 'fields':'{,trackedEntity,programStage,dataValues=[dataElement,value]}', 'order':f'{first_consultation_element_id}:asc', 'pageSize':'1'})
    add_consultation(patient, first_consultation)

    # get last pickup info
    pharmacy_stage = 'P2kauR0fz6C'
    last_pickup_data_element_id = 'f1E1lgVGby4'
    last_pharmacy = api.get('tracker/events', params={'orgUnit':'UnCWi0cmwXy', 'program':program, 'programStage':pharmacy_stage, 'trackedEntity':tracked_entity, 'fields':'{,trackedEntity,programStage,dataValues=[dataElement,value]}', 'order':f'{last_pickup_data_element_id}:desc', 'pageSize':'1'})
    add_pharmacy(patient, last_pharmacy)

    # get last lab info
    lab_stage = 'KXYMcOQZXLf'
    last_lab = api.get('tracker/events', params={'orgUnit':'UnCWi0cmwXy', 'program':program, 'programStage':lab_stage, 'trackedEntity':tracked_entity, 'fields':'{,trackedEntity,programStage,dataValues=[dataElement,value]}', 'order':'occurredAt:desc', 'pageSize':'1'})
    add_laboratory(patient, last_lab)

    # get final outcome
    final_outcome_stage = 'LsiTuPWVsOC'
    final_outcome = api.get('tracker/events', params={'skipPaging':'true','orgUnit':'UnCWi0cmwXy', 'program':program, 'programStage':lab_stage, 'trackedEntity':tracked_entity, 'fields':'{,trackedEntity,programStage,dataValues=[dataElement,value]}'})
    add_final_outcome(patient, final_outcome)

In [6]:
end_period = '2024-03-31'

from src.main.application import *
tx_curr = ComputeTxCurrUseCase()

tx_curr.compute(patients, end_period)

In [7]:
start_period = '2024-01-01'

tx_new = ComputeTxNewUseCase()
tx_new.compute(patients,start_period,end_period)

In [8]:
data = pd.json_normalize(patients)
data[data['txNew']== True]

,enrollment,trackedEntity,program,status,enrolledAt,patientIdentifier,patientAge,patientName,patientSex,artStartDate,firstConsultationDate,pickupQuantity,lastPickupDate,nextPickupDate,txCurr,txNew,lastCD4
29,ELoBtYLF39m,nyRrDEmcTQN,YKs1kXOLb0u,ACTIVE,2024-03-11T00:00:00.000,1609800880/2024T/00032,2000-03-28,Luisa Francisco,Feminino,2024-03-11,2024-03-11,30,2024-03-11,2024-04-10,True,True,NaN
45,hrqSibZ84FV,ow8dJtU54XA,YKs1kXOLb0u,ACTIVE,2024-03-07T00:00:00.000,1609800880/2024T/00031,2002-03-03,Cecilia Shijoleni,Feminino,2024-03-07,2024-03-07,30,2024-03-07,2024-04-06,True,True,NaN
46,A1tbIAUy6as,ekIKK9aH6KZ,YKs1kXOLb0u,ACTIVE,2024-03-14T00:00:00.000,1609800880/2024T/00034,1995-02-06,Eluesa Miguel,Feminino,2024-03-14,2024-03-14,30,2024-03-14,2024-04-13,True,True,NaN
76,cMNK7DQrIDm,FD0PqwaDZpj,YKs1kXOLb0u,ACTIVE,2024-03-07T00:00:00.000,1609800880/2023T/00016,1987-02-07,Ziomana Manuel,Masculino,2024-02-23,2023-02-23,180,2024-03-07,2024-09-03,True,True,NaN
87,Ktnk4YNW6w7,YRU9mvLhC4u,YKs1kXOLb0u,ACTIVE,2024-03-18T00:00:00.000,1609800880/2024T/00035,1988-03-25,Paulino Ndeumonamo,Masculino,2024-03-18,2024-03-18,30,2024-03-18,2024-04-17,True,True,NaN
